# 01. Project Setup and Data Audit

**Project:** EEG Analysis for ASD Identification
| **Author:** Bruno Oswando
| **Date:** 2026

## 🎯 Objective
This notebook initiates the research pipeline by:
1.  Configuring the computational environment (libraries and paths).
2.  Loading metadata and verifying raw data integrity.
3.  **Auditing Events:** Scanning all `.edf` files to ensure they contain the necessary markers ('FACE', 'BASAL') before processing.

## ⚙️ Requirements
* Raw data must be located in `../data/raw/`.

In [2]:
# 1. SETUP AMBIENTE
import os
import sys
import numpy as np
import pandas as pd
import mne
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configuração Visual
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set_theme(style='whitegrid', context='paper', font_scale=1.2)

# Silenciar avisos desnecessários do MNE para limpar o output
mne.set_log_level('WARNING')
warnings.filterwarnings('ignore')

# 2. DEFINIÇÃO DE CONSTANTES
# Seed para garantir reprodutibilidade (resultados iguais)
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Caminhos Relativos
DATA_RAW_PATH = '../data/raw/'
RESULTS_PATH = '../results/'

# Cria pasta de resultados se não existir
if not os.path.exists(RESULTS_PATH):
    os.makedirs(RESULTS_PATH)

print(f"Ambiente Configurado!")
print(f"MNE Version: {mne.__version__}")
print(f"Random Seed: {RANDOM_SEED}")

Ambiente Configurado!
MNE Version: 1.6.0
Random Seed: 42


In [5]:
# 3. AUDITORIA DE EVENTOS (DATA INTEGRITY CHECK)

print(f" Iniciando auditoria em: {os.path.abspath(DATA_RAW_PATH)}")

# Lista arquivos .edf
edf_files = sorted([f for f in os.listdir(DATA_RAW_PATH) if f.lower().endswith('.edf')])
print(f" Total de arquivos encontrados: {len(edf_files)}")

audit_log = []

for filename in edf_files:
    filepath = os.path.join(DATA_RAW_PATH, filename)
    
    try:
        # Lê apenas o cabeçalho
        raw = mne.io.read_raw_edf(filepath, preload=False, verbose=False)
        
        # Extrai anotações
        events, event_id = mne.events_from_annotations(raw, verbose=False)
        
        # Checagem dos marcadores
        has_face = any('FACE' in k.upper() or 'FF' in k or 'FN' in k or 'FR' in k for k in event_id.keys())
        has_basal = any('BASAL' in k.upper() for k in event_id.keys())
        
        audit_log.append({
            'Arquivo': filename,
            'Status': 'OK',
            'N_Eventos': len(events),
            'Tem_Faces': has_face,
            'Tem_Basal': has_basal,
            'Tipos_Encontrados': list(event_id.keys())
        })
        
    except Exception as e:
        audit_log.append({
            'Arquivo': filename,
            'Status': 'FALHA',
            'N_Eventos': 0,
            'Tem_Faces': False,
            'Tem_Basal': False,
            'Tipos_Encontrados': str(e)
        })

# Cria DataFrame e Mostra
df_audit = pd.DataFrame(audit_log)

# Salva o relatório
df_audit.to_csv(os.path.join(RESULTS_PATH, 'relatorio_auditoria_inicial.csv'), index=False)

print("\n Resumo da Auditoria:")
display(df_audit.head())

# Verificação rápida de sucesso
if df_audit['Status'].eq('OK').all():
    print("\n✅ Sucesso! Todos os arquivos estão íntegros.")
else:
    print(f"\n⚠️ Atenção: {len(df_audit[df_audit['Status'] != 'OK'])} arquivos com problemas.")

 Iniciando auditoria em: /workspaces/eeg-asd-classification/data/raw
 Total de arquivos encontrados: 0

 Resumo da Auditoria:


""


KeyError: 'Status'